In [ ]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_excel("senti_data.xlsx")

df.head()

,Bengali,English,Sentiment
0,এর গুপ্তচর,its spy,neutral
1,মল,mall,neutral
2,যখন নতুন চলচ্চিত্রের চেয়ে অনেক বেশি এযেচত্চিন...,while the new film is much more eye-catching t...,positive
3,একটি বিশেষ গুস্সিএদ্ অফ়্তের্স্চূল্ সাথে অভিনব...,an after-school special gussied up with some f...,neutral
4,অম্লীয়,acidic,neutral


In [ ]:
senti_data=df[['Bengali','Sentiment']]
senti_data.head()

,Bengali,Sentiment
0,এর গুপ্তচর,neutral
1,মল,neutral
2,যখন নতুন চলচ্চিত্রের চেয়ে অনেক বেশি এযেচত্চিন...,positive
3,একটি বিশেষ গুস্সিএদ্ অফ়্তের্স্চূল্ সাথে অভিনব...,neutral
4,অম্লীয়,neutral


In [ ]:
senti_data=senti_data[senti_data['Sentiment']!='neutral']
print(senti_data.shape)

(80044, 2)


In [ ]:
list1=senti_data['Sentiment'].tolist()
len(list1)

80044

In [ ]:
l=[]
for i in range(len(list1)):
  if list1[i]=='positive':
    l.append([int(0)])
  elif  list1[i]=='negative':
      l.append([int(1)])
  else:
      l.append([int(2)])

print(l)

[[0], [1], [0], [0], [1], [1], [1], [0], [0], [1], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [1], [1], [0], [1], [0], [0], [0], [1], [0], [1], [0], [1], [0], [1], [0], [1], [1], [0], [1], [1], [0], [0], [1], [1], [0], [1], [1], [1], [0], [0], [1], [0], [0], [1], [0], [1], [1], [0], [1], [0], [1], [0], [1], [0], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [0], [1], [1], [1], [0], [1], [0], [1], [1], [1], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [0], [0], [0], [0], [1], [1], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [0], [1], [0], [1], [0], [0], [1], [0], [0], [1], [1], [0], [0], [1], [1], [1], [1], [1], [1], [0], [1], [1], [0], [0], [1], [1], [1], [0], [1], [0], [1], [1], [1], [1], [0], [0], [1], [1], [1], [0], [1], [0], [0], [0], [1], [0], [0], [1], [0], [0], [0], [0], [1], [1], [0], [0], [0], [1], [1], [0], [0], [0], [1], [0], [0], [0], [1], [0], [1], [0],

In [ ]:
import numpy as np
pad=np.array(l)
print(pad.shape)

(80044, 1)


In [ ]:
np.save('Labels.npy',pad)

In [ ]:
list2=senti_data.Bengali.tolist()
list2

['যখন নতুন চলচ্চিত্রের চেয়ে অনেক বেশি এযেচত্চিন্গ্ এর ব্লূদ্দ্রেন্চেদ্ স্টিফেন নোর্রিন্গ্তোন্দিরেচ্তেদ্ পূর্বসূরী  নতুন চিত্রনাট্য এই ডেভিড ।',
 'প্রায় দেড় ডজন তরুণ তুরস্কের অন্গ্লিন্গ্ কীভাবে কাজ করে যেতে অনেক বার এই  মৌলবাদী  বা  স্তন্যপান  একটি বাক্য',
 'গ্রস্পেদ্ তার',
 'ভালবাসে বালতির দৃষ্টান্ত যে  তার চরিত্র এবং চোম্মুনিচতেস্ সুন্দর কিছু নয়  বরং মানব প্রকৃতির ।',
 'হয় এবং তেদিওউস্ শ্রেণীকক্ষে খেলার মতো কলেজ ইতিহাস কোর্স',
 'ঢুকে যাচ্ছিল উনোব্ত্রুসিবেল্য্ মধ্যে',
 'জচ্কস্স্ একটি অশ্লীল এবং',
 'এমনকি যখন বিদেশী জিনিস পরিচালক    ধার থেকে হলিউড  তারা কথা শেক আপ এবং আরও আকর্ষণীয় ।',
 'দ্য একজন ক্লান্ত দেসিচ্চতেদ্ প্রতিভা  যিনি তাঁর ন্ত্ থেকে ।',
 'পপকর্ন সন্ত্রাসবাদ মাত্রাতিরিক্ত আসল ।',
 'গুজবাম্পস এর উন্চন্ন্য্ গল্প  সাম্প্রদায়িক গ্রাস করে  এবং ন্যায়বিচার',
 'এটি এখনও একটি স্ ছাড়াও ক্রমবর্ধমান তোপ পোস্ত্সবিন্গ্ বেসরকারি রায়ান শ্রদ্ধা সর্বশ্রেষ্ঠ ।',
 'এ ছোট রত্ন বেলজিয়াম',
 'যদি এই ধারণাটি একটি দুর্বল এক স্ না করে  চার্লি সোর্র্য্ সিনেমা ।',
 'যাঁরা প্রেমের ছায়াছবি পরদিসো 

In [ ]:
unique=set(list2)
len(unique)

74443

In [ ]:
sentences=senti_data['Bengali'].astype("str")

In [ ]:
tokenizer = Tokenizer(num_words=74443,lower=True)

tokenizer.fit_on_texts(sentences)

seq_token = tokenizer.texts_to_sequences(sentences)

padded_sequence = pad_sequences(seq_token, maxlen=200) #used to ensure that all sequences in a list have the same length
X= pad_sequences(seq_token)

In [ ]:
X.shape

(80044, 45)

In [ ]:
np.save('Sentences.npy',X)

In [ ]:
import tensorflow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

In [ ]:
sentencess=np.load('Sentences.npy')
label=np.load('Labels.npy')

train_x=X[:78500]
train_y=label[:78500]
test_x=X[78500:]
test_y=label[78500:]

In [ ]:
inp=Input(shape=(45,))
emb=Embedding(np.amax(train_x)+1,128,mask_zero=True)(inp)
lstm1=Bidirectional(LSTM(64,return_sequences=True))(emb)
lstm2=Bidirectional(LSTM(32))(lstm1)
out=Dense(np.amax(label)+1, activation='softmax')(lstm2)

model=Model(inp,out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
es=EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
mc=ModelCheckpoint('checkpoint.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

model.fit(train_x, train_y, batch_size=32, epochs=20,verbose=1, validation_split=0.2, callbacks=[es,mc])
model.save('model.h5')

Epoch 1/20
1963/1963 [==============================] - ETA: 0s - loss: 0.4311 - accuracy: 0.8034
Epoch 1: val_loss improved from inf to 0.33322, saving model to checkpoint.h5
1963/1963 [==============================] - 838s 418ms/step - loss: 0.4311 - accuracy: 0.8034 - val_loss: 0.3332 - val_accuracy: 0.8606
Epoch 2/20
1963/1963 [==============================] - ETA: 0s - loss: 0.2285 - accuracy: 0.9098
Epoch 2: val_loss improved from 0.33322 to 0.30528, saving model to checkpoint.h5
1963/1963 [==============================] - 815s 415ms/step - loss: 0.2285 - accuracy: 0.9098 - val_loss: 0.3053 - val_accuracy: 0.8808
Epoch 3/20
1963/1963 [==============================] - ETA: 0s - loss: 0.1439 - accuracy: 0.9451
Epoch 3: val_loss did not improve from 0.30528
1963/1963 [==============================] - 806s 411ms/step - loss: 0.1439 - accuracy: 0.9451 - val_loss: 0.3313 - val_accuracy: 0.8852
Epoch 4/20
1963/1963 [==============================] - ETA: 0s - loss: 0.1039 - accurac

In [ ]:
model=load_model('checkpoint.h5')
prediction=model.predict(test_x)
y_pred=[]
for i in prediction:
  y_pred.append(np.argmax(i))
y_test=test_y.flatten()
from sklearn.metrics import classification_report, confusion_matrix
print (confusion_matrix(y_test, y_pred))
print (classification_report(y_test, y_pred))

[[684 103]
 [ 89 668]]
              precision    recall  f1-score   support

           0       0.88      0.87      0.88       787
           1       0.87      0.88      0.87       757

    accuracy                           0.88      1544
   macro avg       0.88      0.88      0.88      1544
weighted avg       0.88      0.88      0.88      1544



In [ ]:
y_pred

In [ ]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=45)
    prediction = model.predict(tw)
    y1=[]
    for i in prediction:
      y1.append(np.argmax(i))
    if y1==[1]:
      print("Predicted label: ", "Negative")
    else:
      print("Predicted label: ", "Positive")



test_sentence1 = "হতাশ কারণ প্রতিটি ওবের্ব্রোউঘ্ত্ নতুন অনুক্রম"
predict_sentiment(test_sentence1)

test_sentence2 = "জন্য প্রয়োজন এক সমৃদ্ধ এবং পুরো জীবন"
predict_sentiment(test_sentence2)

test_sentence3="আমি একটি ভালো মেয়ে"
predict_sentiment(test_sentence3)

test_sentence4="আমি একটা খারাপ মেয়ে"
predict_sentiment(test_sentence4)

Predicted label:  Negative
Predicted label:  Positive
Predicted label:  Positive
Predicted label:  Negative


In [ ]:
test_sentence6 ="তারা যোগ অনুশীলন করে না।	"
predict_sentiment(test_sentence6)

test_sentence7 ="তারা জুজু খেলতেন না	"
predict_sentiment(test_sentence7)

test_sentence8 ="তিনি চলে গেলে আমরা দুঃখ পাইনি।	" #negative
predict_sentiment(test_sentence8)

test_sentence9 ="আমি ইংল্যান্ডে উড়ে যাচ্ছি না।	"
predict_sentiment(test_sentence9)

test_sentence10 ="সে সাদা ভাত খাচ্ছিল না।	"  #negative
predict_sentiment(test_sentence10)

Predicted label:  Negative
Predicted label:  Negative
Predicted label:  Positive
Predicted label:  Positive
Predicted label:  Negative


In [ ]:
test_sentence11 ="মারি একজন প্রকাশিত লেখক	"
predict_sentiment(test_sentence11)

test_sentence12 ="তিন বছরে সবাই খুশি হবে"
predict_sentiment(test_sentence12)

test_sentence13 ="নোরা রবার্টস বিশ্বের সবচেয়ে জনপ্রিয় রোম্যান্স লেখক	" 
predict_sentiment(test_sentence13)

test_sentence14 ="আপনি যদি নক্সভিলে চলে যান, আপনি রালা নামে একটি দোকান পাবেন"
predict_sentiment(test_sentence14)

test_sentence15 ="আজ ভুতে ভরে গেছে।" 
predict_sentiment(test_sentence15)

Predicted label:  Positive
Predicted label:  Positive
Predicted label:  Positive
Predicted label:  Negative
Predicted label:  Positive
